In [1]:
from estios.calc import regional_io_projection
from estios.models import InterRegionInputOutput
from estios.uk.scenarios import baseline_england_annual_projection,THREE_UK_CITY_REGIONS
from estios.uk.scenarios import baseline_england_annual_population_projection_config
from estios.uk.ons_population_projections import ONS_ENGLAND_POPULATION_META_DATA,ONSPopulationProjection
from copy import deepcopy
from estios.uk.utils import PUASManager, generate_uk_puas,generate_base_regions
from estios.utils import CITY_COLUMN, OTHER_CITY_COLUMN, generate_i_m_index, generate_ij_index,generate_ij_m_index
from estios.uk.regions import UK_CITY_REGIONS, UK_EPSG_GEO_CODE
from estios.spatial import sum_for_regions_by_attr
from numpy import exp
from typing import Final, Optional,Iterable
from pandas import DataFrame,Series,read_csv

region=[
 'Barnsley',
 'Bristol',
 'Basildon',
 'Birmingham',
 'Blackpool',
 'Bradford',
 'Burnley',
 'Cambridge',
 'Chatham',
 'Coventry',
 'Crawley',
 'Derby',
 'Doncaster',
 'Exeter',
 'Gloucester',
 'Huddersfield',
 'Hull',
 'Ipswich',
 'Leeds',
 'Leicester',
 'Liverpool',
 'London',
 'Luton',
 'Manchester',
 'Mansfield',
 'Middlesbrough',
 'Milton Keynes',
 'Newcastle',
 'Norwich',
 'Nottingham',
 'Oxford',
 'Peterborough',
 'Plymouth',
 'Portsmouth',
 'Preston',
 'Reading',
 'Sheffield',
 'Slough',
 'Southampton',
 'Sunderland',
 'Swindon',
 'Telford',
 'Wakefield',
 'Warrington',
 'Wigan',
 'Worthing',
 'York',]
annuals=baseline_england_annual_projection(regions=region, years=[2022,2027,2032,2037,2042])
IO=deepcopy(annuals[0])

national_employment: Series = (IO.national_employment-IO.employment_table.sum(axis=0))                            
city_distances=IO.distances
city_employment_2017=IO.employment_table_productivity_adjusted

/opt/homebrew/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/Users/bzhang/Library/CloudStorage/OneDrive-TheAlanTuringInstitute/estios-BZ/estios/calc.py:7: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https:

In [2]:
BETA=2
NATIONAL_COLUMN_NAME="UK"
UK_NATIONAL_DISTANCE=703.5
X_i_m_national=(IO.io_table[IO.sectors].loc["Total Sales"]-IO.X_i_m.sum(axis=0)).astype('float64')
X_i_m_national.name=NATIONAL_COLUMN_NAME
x_i_m_summed_national = X_i_m_national*(IO.technical_coefficients.T).sum().astype('float64')
x_i_m_summed_national.name = NATIONAL_COLUMN_NAME
F_i_m_national=IO.F_i_m_national
E_i_m_national=IO.io_table.loc[IO.sectors, IO.export_column_names].sum(axis=1)-IO.E_i_m.sum(axis=0)
E_i_m_national.name="UK"
M_i_m_national=IO.M_i_m_national
national_population=IO.national_population-IO.regional_populations.sum()

In [3]:
def A_i_m_cal(
    city_distances: DataFrame,
    city_employment: DataFrame,
    city_population: Series,
    B_j_m_old=1,
    beta: float = BETA,                            
    include_national: bool = False,                            
    national_column_name: str = NATIONAL_COLUMN_NAME,                            
    national_population: Series = national_population,                            
    national_distance: float = UK_NATIONAL_DISTANCE,                           
) -> DataFrame:
    """Calculate B_j^m via the singly constrained import flow anchor (equation 16)."""
    ijm_index: MultiIndex = generate_ij_m_index(city_employment.index, 
                                               city_employment.columns, 
                                               include_national=include_national)
    A_i_m: DataFrame = DataFrame({'P_i^m': None}, index=ijm_index)
    A_i_m['Distance'] = A_i_m.apply(
        lambda row: 
            city_distances["Distance"][row.name[0]][row.name[1]] 
            if national_column_name not in row.name else national_distance,
        axis=1)
    if include_national:
        city_population=city_population.append(Series([national_population], index=[NATIONAL_COLUMN_NAME]))
    A_i_m['P_i^m'] = A_i_m.apply(
        lambda row: city_population.loc[row.name[1]],
        axis=1)
    A_i_m['c_{ij}^-β'] = A_i_m['Distance'] ** (-1*beta)
    A_i_m["P_i^m * c_{ij}^-β"] = A_i_m["P_i^m"] * A_i_m['c_{ij}^-β']
    A_i_m["P_i^m * c_{ij}^-β"] = A_i_m.groupby(['City', 'Sector'])["P_i^m * c_{ij}^-β"].transform("sum")
    A_i_m["B_j^m"]=B_j_m_old
    A_i_m["B_j^m * sum P_i^m *  c_{ij}^-β"] = A_i_m["P_i^m * c_{ij}^-β"]*A_i_m["B_j^m"]

    # Equation 16
    A_i_m["A_i^m"] = 1/A_i_m["B_j^m * sum P_i^m *  c_{ij}^-β"]
    return A_i_m

def B_j_m_cal(
    city_distances: DataFrame,
    city_employment: DataFrame,
    A_i_m_old=1,
    beta: float = BETA,                            
    include_national: bool = False,                            
    national_column_name: str = NATIONAL_COLUMN_NAME,                            
    national_employment: Series = national_employment,                            
    national_distance: float = UK_NATIONAL_DISTANCE,                           
) -> DataFrame:
    """Calculate B_j^m via the singly constrained import flow anchor (equation 16)."""
    ijm_index: MultiIndex = generate_ij_m_index(city_employment.index, 
                                               city_employment.columns, 
                                               include_national=include_national)
    B_j_m: DataFrame = DataFrame({'Q_i^m': None}, index=ijm_index)
    B_j_m['Distance'] = B_j_m.apply(
        lambda row: 
            city_distances["Distance"][row.name[0]][row.name[1]] 
            if national_column_name not in row.name else national_distance,
        axis=1)
    B_j_m['Q_i^m'] = B_j_m.apply(
        lambda row: city_employment.loc[row.name[0]][row.name[2]]
        if f'{national_column_name}' != row.name[0] 
        else national_employment[row.name[2]],
        axis=1)
    B_j_m['c_{ij}^-β'] = B_j_m['Distance'] ** (-1*beta)
    B_j_m["Q_i^m * c_{ij}^-β"] = B_j_m["Q_i^m"] * B_j_m['c_{ij}^-β']
    B_j_m["sum Q_i^m * c_{ij}^-β"] = B_j_m.groupby(['Other_City', 'Sector'])["Q_i^m * c_{ij}^-β"].transform("sum")
    B_j_m["A_i^m"]=A_i_m_old
    B_j_m["A_i^m * sum Q_i^m * c_{ij}^-β"] = B_j_m["sum Q_i^m * c_{ij}^-β"]*B_j_m["A_i^m"]


    # Equation 16
    B_j_m["B_j^m"] = 1/B_j_m["A_i^m * sum Q_i^m * c_{ij}^-β"]
    return B_j_m

In [4]:
def iteration_for_AiBj(
    city_distances: DataFrame,
    city_employment: DataFrame,
    city_population: Series,
    beta: float = BETA,                            
    include_national: bool = True,                            
    national_column_name: str = NATIONAL_COLUMN_NAME,                            
    national_employment: Series = national_employment,                            
    national_distance: float = UK_NATIONAL_DISTANCE,
    iteration_number: float=20
):
    A_i_m_init= A_i_m_cal(city_distances=city_distances,city_employment=city_employment,city_population=city_population,B_j_m_old=1,beta=beta,include_national=include_national)
    A_i_m_res=A_i_m_init["A_i^m"]
    B_j_m_init= B_j_m_cal(city_distances=city_distances,city_employment=city_employment,A_i_m_old=A_i_m_res,beta=beta,include_national=include_national)
    B_j_m_res=B_j_m_init["B_j^m"]

    for i in range (0,iteration_number):
        old_value=A_i_m_res*B_j_m_res
        A_i_m= A_i_m_cal(city_distances=city_distances,city_employment=city_employment,city_population=city_population,B_j_m_old=B_j_m_res,beta=beta,include_national=include_national)
        A_i_m_res=A_i_m["A_i^m"]
        B_j_m= B_j_m_cal(city_distances=city_distances,city_employment=city_employment,A_i_m_old=A_i_m_res,beta=beta,include_national=include_national)
        B_j_m_res=B_j_m["B_j^m"]
        new_value=A_i_m_res*B_j_m_res
        print(abs(new_value-old_value).sum()/new_value.sum())
        print(A_i_m_res[0])

    return A_i_m,B_j_m


In [5]:
    A_i_m_init= A_i_m_cal(city_distances,city_employment_2017,IO.regional_populations,beta=2,include_national=True)
    A_i_m_res=A_i_m_init["A_i^m"]
    B_j_m_init= B_j_m_cal(city_distances, city_employment_2017,A_i_m_res,beta=2,include_national=True)
    B_j_m_res=B_j_m_init["B_j^m"]

/var/folders/nh/shb6sl_d0p7fsn31td_ttpzc0000gr/T/ipykernel_73398/2872743285.py:23: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  city_population=city_population.append(Series([national_population], index=[NATIONAL_COLUMN_NAME]))


In [6]:
city_employment_2017

,Agriculture,Production,Construction,"Distribution, transport, hotels and restaurants",Information and communication,Financial and insurance,Real estate,Professional and support activities,"Government, health & education",Other services
Area,,,,,,,,,,
Barnsley,23.126825,3379.376685,1662.244916,7.664966e+03,312.885986,353.140787,720.837279,4.345860e+03,1.204954e+04,1023.852654
Bristol,18.519093,14340.238155,12423.257784,6.646806e+04,13362.525481,16118.567537,3463.316609,5.307002e+04,7.810949e+04,9662.668401
Basildon,4.228683,5962.612930,2836.749037,1.599784e+04,2288.417335,1937.127745,553.573963,8.343613e+03,1.217022e+04,1805.229074
Birmingham,132.352655,40387.831458,15220.595250,1.104136e+05,11937.448022,16167.933299,8250.568516,7.958704e+04,1.383546e+05,16741.179664
Blackpool,63.475459,3881.620387,1419.385509,1.559875e+04,954.186865,1356.955945,633.106902,5.367575e+03,2.267793e+04,4014.479570
Bradford,23.071073,17438.528134,3684.972780,2.685596e+04,3468.130147,4227.473669,898.874450,1.445128e+04,3.358106e+04,2553.461144
Burnley,7.746954,5077.534673,928.022994,7.327017e+03,558.984562,425.858535,579.513362,3.457437e+03,8.649254e+03,1359.619492
Cambridge,15.225287,4655.246608,1945.458389,2.436918e+04,9154.888366,1627.404004,1660.943052,2.384207e+04,4.558143e+04,4814.585286
Chatham,20.902670,1047.851244,1201.906654,4.258068e+03,603.296172,957.535479,390.907702,2.396025e+03,5.808379e+03,753.529076


In [7]:
IO.regional_populations

Barnsley           243341.0
Bristol            738279.0
Basildon           184479.0
Birmingham        2537154.0
Blackpool          218733.0
Bradford           534800.0
Burnley            178401.0
Cambridge          124919.0
Chatham            277616.0
Coventry           360149.0
Crawley            111664.0
Derby              257034.0
Doncaster          308940.0
Exeter             128916.0
Gloucester         129083.0
Huddersfield       437145.0
Hull               260673.0
Ipswich            138480.0
Leeds              784846.0
Leicester          509552.0
Liverpool          640109.0
London           10062284.0
Luton              214658.0
Manchester        2474149.0
Mansfield          234740.0
Middlesbrough      473131.0
Milton Keynes      267521.0
Newcastle          852289.0
Norwich            268888.0
Nottingham         674369.0
Oxford             154582.0
Peterborough       198914.0
Plymouth           263070.0
Portsmouth         541511.0
Preston            367518.0
Reading            3

In [8]:
 A_i_m=A_i_m_cal(city_distances,city_employment_2017,IO.regional_populations,1,beta=2,include_national=True)

/var/folders/nh/shb6sl_d0p7fsn31td_ttpzc0000gr/T/ipykernel_73398/2872743285.py:23: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  city_population=city_population.append(Series([national_population], index=[NATIONAL_COLUMN_NAME]))


In [9]:
B_j_m

NameError: name 'B_j_m' is not defined

In [10]:
A_i_m,B_j_m=iteration_for_AiBj(city_distances,  city_employment_2017,IO.regional_populations,beta=2,iteration_number=10,    include_national = True)

/var/folders/nh/shb6sl_d0p7fsn31td_ttpzc0000gr/T/ipykernel_73398/2872743285.py:23: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  city_population=city_population.append(Series([national_population], index=[NATIONAL_COLUMN_NAME]))
/var/folders/nh/shb6sl_d0p7fsn31td_ttpzc0000gr/T/ipykernel_73398/2872743285.py:23: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  city_population=city_population.append(Series([national_population], index=[NATIONAL_COLUMN_NAME]))


7.965599845722701e-17
1.0464699515729256e-08


/var/folders/nh/shb6sl_d0p7fsn31td_ttpzc0000gr/T/ipykernel_73398/2872743285.py:23: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  city_population=city_population.append(Series([national_population], index=[NATIONAL_COLUMN_NAME]))


6.183687663129658e-17
1.008090259529017e-12


/var/folders/nh/shb6sl_d0p7fsn31td_ttpzc0000gr/T/ipykernel_73398/2872743285.py:23: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  city_population=city_population.append(Series([national_population], index=[NATIONAL_COLUMN_NAME]))


...

8.363925028067854e-17
8.056294886884852e-37


/var/folders/nh/shb6sl_d0p7fsn31td_ttpzc0000gr/T/ipykernel_73398/2872743285.py:23: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  city_population=city_population.append(Series([national_population], index=[NATIONAL_COLUMN_NAME]))


8.751099378145064e-17
7.760827141911566e-41


/var/folders/nh/shb6sl_d0p7fsn31td_ttpzc0000gr/T/ipykernel_73398/2872743285.py:23: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  city_population=city_population.append(Series([national_population], index=[NATIONAL_COLUMN_NAME]))


8.563351721798464e-17
7.476195791279022e-45


In [11]:
def b_ij_m_cal(
    city_distances: DataFrame,
    city_employment: DataFrame,
    city_population: Series,
    A_i_m: DataFrame,
    B_j_m: DataFrame,
    beta: float = BETA,                            
    include_national: bool = False,                            
    national_column_name: str = NATIONAL_COLUMN_NAME,
    national_population: Series = national_population,                            
    national_employment: Series = national_employment,                            
    national_distance: float = UK_NATIONAL_DISTANCE,                           
) -> DataFrame:
    """Calculate B_j^m via the singly constrained import flow anchor (equation 16)."""
    ijm_index: MultiIndex = generate_ij_m_index(city_employment.index, 
                                               city_employment.columns, 
                                               include_national=include_national)
    b_ij_m: DataFrame = DataFrame({'P_i^m': None}, index=ijm_index)
    b_ij_m['Distance'] = b_ij_m.apply(
        lambda row: 
            city_distances["Distance"][row.name[0]][row.name[1]] 
            if national_column_name not in row.name else national_distance,
        axis=1)
    b_ij_m['c_{ij})^-β'] = b_ij_m['Distance'] ** (-1*beta)
    if include_national:
        city_population=city_population.append(Series([national_population], index=[NATIONAL_COLUMN_NAME]))
    b_ij_m['P_i^m'] = b_ij_m.apply(
        lambda row: city_population.loc[row.name[1]],
        axis=1)
    b_ij_m['Q_i^m'] = b_ij_m.apply(
        lambda row: city_employment.loc[row.name[0]][row.name[2]]
        if f'{national_column_name}' != row.name[0] 
        else national_employment[row.name[2]],
        axis=1)
    b_ij_m["A_i^m"]=A_i_m["A_i^m"]
    b_ij_m["B_j^m"]=B_j_m["B_j^m"]
    b_ij_m["init_b_ij^m"]=b_ij_m["A_i^m"]*b_ij_m["B_j^m"]*b_ij_m['Q_i^m']*b_ij_m['P_i^m']*b_ij_m['c_{ij})^-β']
    b_ij_m["sum_j b_ij^m"] = b_ij_m.groupby(['Other_City', 'Sector'])["init_b_ij^m"].transform("sum")
    b_ij_m["K"]=1/b_ij_m["sum_j b_ij^m"]
    b_ij_m["b_ij^m"]=b_ij_m["init_b_ij^m"]*b_ij_m["K"]
    return b_ij_m

In [12]:
b_ij_m=b_ij_m_cal(city_distances,  city_employment_2017,IO.regional_populations,A_i_m,B_j_m,beta=2,include_national=True)
b_ij_m

/var/folders/nh/shb6sl_d0p7fsn31td_ttpzc0000gr/T/ipykernel_73398/3282348484.py:26: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  city_population=city_population.append(Series([national_population], index=[NATIONAL_COLUMN_NAME]))


P_i^m  \
City     Other_City Sector                                                      
Barnsley Bristol    Agriculture                                      738279.0   
                    Production                                       738279.0   
                    Construction                                     738279.0   
                    Distribution, transport, hotels and restaurants  738279.0   
                    Information and communication                    738279.0   
...                                                                       ...   
UK       York       Financial and insurance                          208163.0   
                    Real estate                                      208163.0   
                    Professional and support activities              208163.0   
                    Government, health & education                   208163.0   
                    Other services                                   208163.0   

                                                                       Distance  \
City     Other_City Sector                                                        
Barnsley Bristol    Agriculture                                      245.655622   
                    Production                                       245.655622   
                    Construction                                     245.655622   
                    Distribution, transport, hotels and restaurants  245.655622   
                    Information and communication                    245.655622   
...                                                                         ...   
UK       York       Financial and insurance                          703.500000   
                    Real estate                                      703.500000   
                    Professional and support activities              703.500000   
                    Government, health & education                   703.500000   
                    Other services                                   703.500000   

                                                                     c_{ij})^-β  \
City     Other_City Sector                                                        
Barnsley Bristol    Agriculture                                        0.000017   
                    Production                                         0.000017   
                    Construction                                       0.000017   
                    Distribution, transport, hotels and restaurants    0.000017   
                    Information and communication                      0.000017   
...                                                                         ...   
UK       York       Financial and insurance                            0.000002   
                    Real estate                                        0.000002   
                    Professional and support activities                0.000002   
                    Government, health & education                     0.000002   
                    Other services                                     0.000002   

                                                                            Q_i^m  \
City     Other_City Sector                                                          
Barnsley Bristol    Agriculture                                      2.312683e+01   
                    Production                                       3.379377e+03   
                    Construction                                     1.662245e+03   
                    Distribution, transport, hotels and restaurants  7.664966e+03   
                    Information and communication                    3.128860e+02   
...                                                                           ...   
UK       York       Financial and insurance                          5.308500e+05   
                    Real estate                                      3.470000e+05   
        

In [13]:
y_ij_m_new=b_ij_m.reset_index()
y_ij_m_new

,City,Other_City,Sector,P_i^m,Distance,c_{ij})^-β,Q_i^m,A_i^m,B_j^m,init_b_ij^m,sum_j b_ij^m,K,b_ij^m
0,Barnsley,Bristol,Agriculture,738279.0,245.655622,0.000017,2.312683e+01,7.476196e-45,1.508345e+44,319.054274,738279.0,0.000001,0.000432
1,Barnsley,Bristol,Production,738279.0,245.655622,0.000017,3.379377e+03,2.129867e-29,1.507862e+27,1327.755052,738279.0,0.000001,0.001798
2,Barnsley,Bristol,Construction,738279.0,245.655622,0.000017,1.662245e+03,2.145780e-31,2.370314e+29,1034.315682,738279.0,0.000001,0.001401
3,Barnsley,Bristol,"Distribution, transport, hotels and restaurants",738279.0,245.655622,0.000017,7.664966e+03,1.306954e-23,6.481316e+20,794.329687,738279.0,0.000001,0.001076
4,Barnsley,Bristol,Information and communication,738279.0,245.655622,0.000017,3.128860e+02,3.901782e-30,9.753494e+27,145.672115,738279.0,0.000001,0.000197
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22555,UK,York,Financial and insurance,208163.0,703.500000,0.000002,5.308500e+05,2.063553e-05,1.576030e+03,7261.517635,208163.0,0.000005,0.034884
22556,UK,York,Real estate,208163.0,703.500000,0.000002,3.470000e+05,5.347316e-10,1.748608e+08,13646.869313,208163.0,0.000005,0.065559
22557,UK,York,Professional and support activities,208163.0,703.500000,0.000002,3.653200e+06,7.266780e+01,9.912098e-05,11067.674742,208163.0,0.000005,0.053168
22558,UK,York,"Government, health & education",208163.0,703.500000,0.000002,5.808000e+06,5.519058e+03,8.464287e-07,11411.881553,208163.0,0.000005,0.054822


In [14]:
k=1/y_ij_m_new[y_ij_m_new["Sector"]=="Agriculture"].groupby("City")["init_b_ij^m"].agg("sum")
k.name="K_new"
Agriculture_flow=y_ij_m_new[y_ij_m_new["Sector"]=="Agriculture"]
Agriculture_flow=Agriculture_flow.join(k,on="City")
Agriculture_flow["b_ij^m"]=Agriculture_flow["init_b_ij^m"]*Agriculture_flow["K_new"]
Mim_Agriculture=IO.E_i_m["Agriculture"].append(Series(E_i_m_national["Agriculture"], index=[NATIONAL_COLUMN_NAME]))
Mim_Agriculture.name="M_i^m"
Agriculture_flow=Agriculture_flow.join(Mim_Agriculture,on="City")
Agriculture_flow["y_ij^m"]=Agriculture_flow["b_ij^m"]*Agriculture_flow["M_i^m"]*0.5
y_ij_m_new.loc[Agriculture_flow.index,["y_ij^m"]]=Agriculture_flow["y_ij^m"]
(Agriculture_flow.groupby("City")["y_ij^m"].agg("sum")-Agriculture_flow.groupby("Other_City")["y_ij^m"].agg("sum"))

/var/folders/nh/shb6sl_d0p7fsn31td_ttpzc0000gr/T/ipykernel_73398/2260243597.py:6: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Mim_Agriculture=IO.E_i_m["Agriculture"].append(Series(E_i_m_national["Agriculture"], index=[NATIONAL_COLUMN_NAME]))


City
Barnsley        -8.816353e+06
Basildon        -5.590088e+06
Birmingham      -1.151172e+08
Blackpool       -1.018210e+07
Bradford        -1.946667e+07
Bristol         -3.588172e+07
Burnley         -7.519727e+06
Cambridge       -5.200322e+06
Chatham         -9.539902e+06
Coventry        -1.376440e+07
Crawley         -3.544898e+06
Derby           -1.097063e+07
Doncaster       -1.242965e+07
Exeter          -6.454036e+06
Gloucester      -5.951088e+06
Huddersfield    -1.574821e+07
Hull            -1.255938e+07
Ipswich         -6.379566e+06
Leeds           -3.037678e+07
Leicester       -2.123871e+07
Liverpool       -2.877035e+07
London          -4.692579e+08
Luton           -6.946804e+06
Manchester      -1.082041e+08
Mansfield       -9.289801e+06
Middlesbrough   -2.368272e+07
Milton Keynes   -1.043234e+07
Newcastle       -4.369812e+07
Norwich         -1.312775e+07
Nottingham      -2.459966e+07
Oxford          -6.246352e+06
Peterborough    -8.927364e+06
Plymouth        -1.342865e+07
Ports

In [15]:
k=1/y_ij_m_new[y_ij_m_new["Sector"]=="Production"].groupby("City")["init_b_ij^m"].agg("sum")
k.name="K_new"
Production_flow=y_ij_m_new[y_ij_m_new["Sector"]=="Production"]
Production_flow=Production_flow.join(k,on="City")
Production_flow["b_ij^m"]=Production_flow["init_b_ij^m"]*Production_flow["K_new"]
Mim_Production=IO.E_i_m["Production"].append(Series(E_i_m_national["Production"], index=[NATIONAL_COLUMN_NAME]))
Mim_Production.name="M_i^m"
Production_flow=Production_flow.join(Mim_Production,on="City")
Production_flow["y_ij^m"]=Production_flow["b_ij^m"]*Production_flow["M_i^m"]*0.5
y_ij_m_new.loc[Production_flow.index,["y_ij^m"]]=Production_flow["y_ij^m"]
(Production_flow.groupby("City")["y_ij^m"].agg("sum")-Production_flow.groupby("Other_City")["y_ij^m"].agg("sum"))

/var/folders/nh/shb6sl_d0p7fsn31td_ttpzc0000gr/T/ipykernel_73398/594300100.py:6: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Mim_Production=IO.E_i_m["Production"].append(Series(E_i_m_national["Production"], index=[NATIONAL_COLUMN_NAME]))


City
Barnsley        -1.470716e+08
Basildon        -9.451961e+07
Birmingham      -3.625367e+09
Blackpool       -5.959658e+08
Bradford         2.303289e+07
Bristol         -3.107664e+09
Burnley         -1.817557e+08
Cambridge       -1.845555e+08
Chatham         -4.225649e+08
Coventry         1.562016e+08
Crawley          8.170444e+07
Derby            5.006495e+08
Doncaster       -3.213688e+08
Exeter          -8.623506e+08
Gloucester      -2.028735e+08
Huddersfield    -2.259315e+08
Hull            -5.734433e+08
Ipswich         -5.017984e+08
Leeds           -1.632260e+08
Leicester       -6.193996e+07
Liverpool       -1.291554e+09
London          -2.870735e+10
Luton            1.260009e+06
Manchester      -4.185170e+09
Mansfield       -1.929041e+08
Middlesbrough   -2.278693e+09
Milton Keynes   -2.607737e+08
Newcastle       -3.579241e+09
Norwich         -1.459863e+09
Nottingham      -4.793143e+08
Oxford          -1.710048e+08
Peterborough    -3.572884e+08
Plymouth        -2.577973e+09
Ports

In [16]:
k=1/y_ij_m_new[y_ij_m_new["Sector"]=="Construction"].groupby("City")["init_b_ij^m"].agg("sum")
k.name="K_new"
Construction_flow=y_ij_m_new[y_ij_m_new["Sector"]=="Construction"]
Construction_flow=Construction_flow.join(k,on="City")
Construction_flow["b_ij^m"]=Construction_flow["init_b_ij^m"]*Construction_flow["K_new"]
Mim_Construction=IO.E_i_m["Construction"].append(Series(E_i_m_national["Construction"], index=[NATIONAL_COLUMN_NAME]))
Mim_Construction.name="M_i^m"
Construction_flow=Construction_flow.join(Mim_Construction,on="City")
Construction_flow["y_ij^m"]=Construction_flow["b_ij^m"]*Construction_flow["M_i^m"]*.5
y_ij_m_new.loc[Construction_flow.index,["y_ij^m"]]=Construction_flow["y_ij^m"]
(Construction_flow.groupby("City")["y_ij^m"].agg("sum")-Construction_flow.groupby("Other_City")["y_ij^m"].agg("sum"))

/var/folders/nh/shb6sl_d0p7fsn31td_ttpzc0000gr/T/ipykernel_73398/2040499009.py:6: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Mim_Construction=IO.E_i_m["Construction"].append(Series(E_i_m_national["Construction"], index=[NATIONAL_COLUMN_NAME]))


City
Barnsley        -2.642204e+06
Basildon        -4.129742e+05
Birmingham      -7.713486e+07
Blackpool       -7.878151e+06
Bradford        -4.461360e+06
Bristol         -2.963499e+07
Burnley         -4.109899e+06
Cambridge       -1.647694e+06
Chatham         -2.639386e+06
Coventry        -5.975422e+06
Crawley          1.025822e+05
Derby           -3.965897e+06
Doncaster       -5.706899e+06
Exeter          -8.255543e+06
Gloucester      -2.725209e+06
Huddersfield    -5.558275e+06
Hull            -1.127068e+07
Ipswich         -3.424700e+06
Leeds           -7.768441e+06
Leicester       -1.106439e+07
Liverpool       -1.858163e+07
London          -3.594642e+08
Luton           -4.681389e+05
Manchester      -6.041971e+07
Mansfield       -4.475212e+06
Middlesbrough   -2.888139e+07
Milton Keynes   -3.092340e+06
Newcastle       -4.580810e+07
Norwich         -1.293319e+07
Nottingham      -1.402601e+07
Oxford           4.557733e+05
Peterborough    -5.175111e+06
Plymouth        -2.418953e+07
Ports

In [17]:
k=1/y_ij_m_new[y_ij_m_new["Sector"]=="Distribution, transport, hotels and restaurants"].groupby("City")["init_b_ij^m"].agg("sum")
k.name="K_new"
DTHR_flow=y_ij_m_new[y_ij_m_new["Sector"]=="Distribution, transport, hotels and restaurants"]
DTHR_flow=DTHR_flow.join(k,on="City")
DTHR_flow["b_ij^m"]=DTHR_flow["init_b_ij^m"]*DTHR_flow["K_new"]
Mim_DTHR=IO.E_i_m["Distribution, transport, hotels and restaurants"].append(Series(E_i_m_national["Distribution, transport, hotels and restaurants"], index=[NATIONAL_COLUMN_NAME]))
Mim_DTHR.name="M_i^m"
DTHR_flow=DTHR_flow.join(Mim_DTHR,on="City")
DTHR_flow["y_ij^m"]=DTHR_flow["b_ij^m"]*DTHR_flow["M_i^m"]*.5
y_ij_m_new.loc[DTHR_flow.index,["y_ij^m"]]=DTHR_flow["y_ij^m"]
(DTHR_flow.groupby("City")["y_ij^m"].agg("sum")-DTHR_flow.groupby("Other_City")["y_ij^m"].agg("sum"))

/var/folders/nh/shb6sl_d0p7fsn31td_ttpzc0000gr/T/ipykernel_73398/687137533.py:6: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Mim_DTHR=IO.E_i_m["Distribution, transport, hotels and restaurants"].append(Series(E_i_m_national["Distribution, transport, hotels and restaurants"], index=[NATIONAL_COLUMN_NAME]))


City
Barnsley        -1.011961e+08
Basildon        -1.121016e+07
Birmingham      -2.463203e+09
Blackpool       -2.508977e+08
Bradford        -1.304310e+08
Bristol         -1.120899e+09
Burnley         -1.326660e+08
Cambridge        1.868439e+07
Chatham         -1.380639e+08
Coventry        -1.089837e+08
Crawley          1.626991e+08
Derby           -6.337930e+07
Doncaster       -2.049285e+08
Exeter          -3.425956e+08
Gloucester      -8.878387e+07
Huddersfield    -1.871177e+08
Hull            -4.039339e+08
Ipswich         -1.021081e+08
Leeds           -1.795314e+08
Leicester       -3.197043e+08
Liverpool       -5.449427e+08
London          -8.602735e+09
Luton            4.004165e+07
Manchester      -2.195497e+09
Mansfield       -1.569874e+08
Middlesbrough   -1.061478e+09
Milton Keynes    2.072040e+07
Newcastle       -1.607629e+09
Norwich         -4.634497e+08
Nottingham      -4.245342e+08
Oxford          -2.172476e+07
Peterborough    -1.004714e+08
Plymouth        -1.089654e+09
Ports

In [18]:
k=1/y_ij_m_new[y_ij_m_new["Sector"]=="Information and communication"].groupby("City")["init_b_ij^m"].agg("sum")
k.name="K_new"
Information_flow=y_ij_m_new[y_ij_m_new["Sector"]=="Information and communication"]
Information_flow=Information_flow.join(k,on="City")
Information_flow["b_ij^m"]=Information_flow["init_b_ij^m"]*Information_flow["K_new"]
Mim_infomrmation=IO.E_i_m["Information and communication"].append(Series(E_i_m_national["Information and communication"], index=[NATIONAL_COLUMN_NAME]))
Mim_infomrmation.name="M_i^m"
Information_flow=Information_flow.join(Mim_infomrmation,on="City")
Information_flow["y_ij^m"]=Information_flow["b_ij^m"]*Information_flow["M_i^m"]*.5
y_ij_m_new.loc[Information_flow.index,["y_ij^m"]]=Information_flow["y_ij^m"]
(Information_flow.groupby("City")["y_ij^m"].agg("sum")-Information_flow.groupby("Other_City")["y_ij^m"].agg("sum"))

/var/folders/nh/shb6sl_d0p7fsn31td_ttpzc0000gr/T/ipykernel_73398/253290130.py:6: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Mim_infomrmation=IO.E_i_m["Information and communication"].append(Series(E_i_m_national["Information and communication"], index=[NATIONAL_COLUMN_NAME]))


City
Barnsley        -5.831692e+07
Basildon        -1.001329e+07
Birmingham      -8.910053e+08
Blackpool       -1.312996e+08
Bradford        -5.945054e+07
Bristol         -2.537972e+08
Burnley         -6.328613e+07
Cambridge        7.763645e+07
Chatham         -5.278194e+07
Coventry        -6.400395e+07
Crawley          1.220226e+07
Derby           -3.484940e+07
Doncaster       -1.067684e+08
Exeter          -8.403230e+07
Gloucester      -2.937380e+07
Huddersfield    -9.650472e+07
Hull            -1.687368e+08
Ipswich         -3.447510e+07
Leeds           -9.302636e+07
Leicester       -1.208883e+08
Liverpool       -2.816462e+08
London           9.207154e+08
Luton           -1.996518e+07
Manchester      -9.639678e+08
Mansfield       -6.659039e+07
Middlesbrough   -4.056187e+08
Milton Keynes    3.703226e+07
Newcastle       -7.100476e+08
Norwich         -1.277157e+08
Nottingham      -1.561206e+08
Oxford           4.011873e+07
Peterborough    -2.548810e+07
Plymouth        -3.173044e+08
Ports

In [19]:
k=1/y_ij_m_new[y_ij_m_new["Sector"]=="Financial and insurance"].groupby("City")["init_b_ij^m"].agg("sum")
k.name="K_new"
Financial_flow=y_ij_m_new[y_ij_m_new["Sector"]=="Financial and insurance"]
Financial_flow=Financial_flow.join(k,on="City")
Financial_flow["b_ij^m"]=Financial_flow["init_b_ij^m"]*Financial_flow["K_new"]
Mim_Financial=IO.E_i_m["Financial and insurance"].append(Series(E_i_m_national["Financial and insurance"], index=[NATIONAL_COLUMN_NAME]))
Mim_Financial.name="M_i^m"
Financial_flow=Financial_flow.join(Mim_Financial,on="City")
Financial_flow["y_ij^m"]=Financial_flow["b_ij^m"]*Financial_flow["M_i^m"]*0.5
y_ij_m_new.loc[Financial_flow.index,["y_ij^m"]]=Financial_flow["y_ij^m"]
(Financial_flow.groupby("City")["y_ij^m"].agg("sum")-Financial_flow.groupby("Other_City")["y_ij^m"].agg("sum"))

/var/folders/nh/shb6sl_d0p7fsn31td_ttpzc0000gr/T/ipykernel_73398/3073565264.py:6: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Mim_Financial=IO.E_i_m["Financial and insurance"].append(Series(E_i_m_national["Financial and insurance"], index=[NATIONAL_COLUMN_NAME]))


City
Barnsley        -8.539342e+07
Basildon        -2.185647e+07
Birmingham      -1.266863e+09
Blackpool       -1.478101e+08
Bradford        -1.641830e+07
Bristol         -1.213791e+08
Burnley         -8.380087e+07
Cambridge       -1.594384e+07
Chatham         -1.048644e+08
Coventry        -5.744711e+07
Crawley          3.075453e+07
Derby           -1.100990e+08
Doncaster       -1.293414e+08
Exeter          -1.318732e+08
Gloucester       3.722785e+07
Huddersfield    -1.379311e+08
Hull            -2.460925e+08
Ipswich          3.578494e+07
Leeds            2.449696e+08
Leicester       -2.089739e+08
Liverpool       -2.673806e+08
London           2.377980e+09
Luton           -6.512794e+07
Manchester      -1.195500e+09
Mansfield       -1.237604e+08
Middlesbrough   -5.108752e+08
Milton Keynes    1.205113e+08
Newcastle       -5.711806e+08
Norwich         -2.699419e+07
Nottingham      -3.354179e+08
Oxford          -5.178492e+07
Peterborough    -5.498551e+06
Plymouth        -5.011296e+08
Ports

In [20]:
k=1/y_ij_m_new[y_ij_m_new["Sector"]=="Real estate"].groupby("City")["init_b_ij^m"].agg("sum")
k.name="K_new"
estate_flow=y_ij_m_new[y_ij_m_new["Sector"]=="Real estate"]
estate_flow=estate_flow.join(k,on="City")
estate_flow["b_ij^m"]=estate_flow["init_b_ij^m"]*estate_flow["K_new"]
Mim_estate=IO.E_i_m["Real estate"].append(Series(E_i_m_national["Real estate"], index=[NATIONAL_COLUMN_NAME]))
Mim_estate.name="M_i^m"
estate_flow=estate_flow.join(Mim_estate,on="City")
estate_flow["y_ij^m"]=estate_flow["b_ij^m"]*estate_flow["M_i^m"]*0.5
y_ij_m_new.loc[estate_flow.index,["y_ij^m"]]=estate_flow["y_ij^m"]
(estate_flow.groupby("City")["y_ij^m"].agg("sum")-estate_flow.groupby("Other_City")["y_ij^m"].agg("sum"))

/var/folders/nh/shb6sl_d0p7fsn31td_ttpzc0000gr/T/ipykernel_73398/2534988734.py:6: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Mim_estate=IO.E_i_m["Real estate"].append(Series(E_i_m_national["Real estate"], index=[NATIONAL_COLUMN_NAME]))


City
Barnsley        -1.237178e+06
Basildon        -7.677737e+05
Birmingham      -2.878141e+07
Blackpool       -3.413999e+06
Bradford        -2.765156e+06
Bristol         -1.421021e+07
Burnley         -1.426778e+06
Cambridge        9.671547e+05
Chatham         -1.997177e+06
Coventry        -4.283955e+05
Crawley         -3.986432e+05
Derby           -1.861427e+05
Doncaster       -3.754729e+06
Exeter          -3.863492e+06
Gloucester      -1.287276e+06
Huddersfield    -2.556123e+06
Hull            -5.844702e+06
Ipswich         -1.382234e+06
Leeds           -3.042035e+06
Leicester       -4.667129e+06
Liverpool       -5.489620e+06
London          -1.023123e+08
Luton            1.191655e+06
Manchester      -2.625744e+07
Mansfield       -2.468711e+06
Middlesbrough   -1.503970e+07
Milton Keynes    4.274160e+05
Newcastle       -2.587968e+07
Norwich         -5.438884e+06
Nottingham      -5.791842e+06
Oxford           6.579072e+05
Peterborough    -1.768298e+06
Plymouth        -1.337132e+07
Ports

In [21]:
k=1/y_ij_m_new[y_ij_m_new["Sector"]=="Professional and support activities"].groupby("City")["init_b_ij^m"].agg("sum")
k.name="K_new"
Professional_flow=y_ij_m_new[y_ij_m_new["Sector"]=="Professional and support activities"]
Professional_flow=Professional_flow.join(k,on="City")
Professional_flow["b_ij^m"]=Professional_flow["init_b_ij^m"]*Professional_flow["K_new"]
Mim_Professional=IO.E_i_m["Professional and support activities"].append(Series(E_i_m_national["Professional and support activities"], index=[NATIONAL_COLUMN_NAME]))
Mim_Professional.name="M_i^m"
Professional_flow=Professional_flow.join(Mim_Professional,on="City")
Professional_flow["y_ij^m"]=Professional_flow["b_ij^m"]*Professional_flow["M_i^m"]*0.5
y_ij_m_new.loc[Professional_flow.index,["y_ij^m"]]=Professional_flow["y_ij^m"]
(Professional_flow.groupby("City")["y_ij^m"].agg("sum")-Professional_flow.groupby("Other_City")["y_ij^m"].agg("sum"))

/var/folders/nh/shb6sl_d0p7fsn31td_ttpzc0000gr/T/ipykernel_73398/876884448.py:6: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Mim_Professional=IO.E_i_m["Professional and support activities"].append(Series(E_i_m_national["Professional and support activities"], index=[NATIONAL_COLUMN_NAME]))


City
Barnsley        -9.842100e+07
Basildon        -3.061525e+07
Birmingham      -2.027683e+09
Blackpool       -2.450251e+08
Bradford        -1.153801e+08
Bristol         -7.873602e+08
Burnley         -1.162437e+08
Cambridge        9.435482e+07
Chatham         -1.368616e+08
Coventry        -8.551202e+07
Crawley          1.000927e+08
Derby           -6.765665e+07
Doncaster       -2.039227e+08
Exeter          -2.556808e+08
Gloucester      -7.455200e+07
Huddersfield    -1.813257e+08
Hull            -3.411422e+08
Ipswich         -7.825836e+07
Leeds           -3.927044e+06
Leicester       -2.761418e+08
Liverpool       -3.931157e+08
London          -3.726238e+09
Luton            5.942712e+07
Manchester      -1.544817e+09
Mansfield       -1.385108e+08
Middlesbrough   -9.563805e+08
Milton Keynes    3.978861e+07
Newcastle       -1.696165e+09
Norwich         -3.463772e+08
Nottingham      -3.793944e+08
Oxford           9.347418e+06
Peterborough    -2.613303e+07
Plymouth        -8.986635e+08
Ports

In [22]:
k=1/y_ij_m_new[y_ij_m_new["Sector"]=="Government, health & education"].groupby("City")["init_b_ij^m"].agg("sum")
k.name="K_new"
education_flow=y_ij_m_new[y_ij_m_new["Sector"]=="Government, health & education"]
education_flow=education_flow.join(k,on="City")
education_flow["b_ij^m"]=education_flow["init_b_ij^m"]*education_flow["K_new"]
Mim_education=IO.E_i_m["Government, health & education"].append(Series(E_i_m_national["Government, health & education"], index=[NATIONAL_COLUMN_NAME]))
Mim_education.name="M_i^m"
education_flow=education_flow.join(Mim_education,on="City")
education_flow["y_ij^m"]=education_flow["b_ij^m"]*education_flow["M_i^m"]*0.5
y_ij_m_new.loc[education_flow.index,["y_ij^m"]]=education_flow["y_ij^m"]
(education_flow.groupby("City")["y_ij^m"].agg("sum")-education_flow.groupby("Other_City")["y_ij^m"].agg("sum"))

/var/folders/nh/shb6sl_d0p7fsn31td_ttpzc0000gr/T/ipykernel_73398/1798056449.py:6: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Mim_education=IO.E_i_m["Government, health & education"].append(Series(E_i_m_national["Government, health & education"], index=[NATIONAL_COLUMN_NAME]))


City
Barnsley        -6.581037e+06
Basildon        -3.976215e+06
Birmingham      -2.149546e+08
Blackpool       -1.703058e+07
Bradford        -6.866311e+06
Bristol         -1.073604e+08
Burnley         -1.122997e+07
Cambridge        1.529577e+07
Chatham         -1.544150e+07
Coventry        -1.935521e+06
Crawley         -2.427899e+05
Derby            1.032222e+06
Doncaster       -1.616435e+07
Exeter          -2.812867e+07
Gloucester      -5.692903e+06
Huddersfield    -1.602033e+07
Hull            -3.501710e+07
Ipswich         -9.558805e+06
Leeds           -1.425264e+06
Leicester       -1.722377e+07
Liverpool       -4.058180e+07
London          -1.058604e+09
Luton            1.172943e+06
Manchester      -2.055855e+08
Mansfield       -1.397875e+07
Middlesbrough   -8.836867e+07
Milton Keynes   -2.170120e+06
Newcastle       -1.247376e+08
Norwich         -4.803633e+07
Nottingham      -2.990459e+07
Oxford           2.263064e+07
Peterborough    -1.255629e+07
Plymouth        -1.034080e+08
Ports

In [23]:
k=1/y_ij_m_new[y_ij_m_new["Sector"]=="Other services"].groupby("City")["init_b_ij^m"].agg("sum")
k.name="K_new"
other_flow=y_ij_m_new[y_ij_m_new["Sector"]=="Other services"]
other_flow=other_flow.join(k,on="City")
other_flow["b_ij^m"]=other_flow["init_b_ij^m"]*other_flow["K_new"]
Mim_other=IO.E_i_m["Other services"].append(Series(E_i_m_national["Other services"], index=[NATIONAL_COLUMN_NAME]))
Mim_other.name="M_i^m"
other_flow=other_flow.join(Mim_other,on="City")
other_flow["y_ij^m"]=other_flow["b_ij^m"]*other_flow["M_i^m"]*0.5
y_ij_m_new.loc[other_flow.index,["y_ij^m"]]=other_flow["y_ij^m"]
(other_flow.groupby("City")["y_ij^m"].agg("sum")-other_flow.groupby("Other_City")["y_ij^m"].agg("sum"))

/var/folders/nh/shb6sl_d0p7fsn31td_ttpzc0000gr/T/ipykernel_73398/1667241145.py:6: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Mim_other=IO.E_i_m["Other services"].append(Series(E_i_m_national["Other services"], index=[NATIONAL_COLUMN_NAME]))


City
Barnsley        -7.786986e+06
Basildon        -2.325742e+06
Birmingham      -1.621905e+08
Blackpool       -1.329251e+07
Bradford        -1.160059e+07
Bristol         -7.098336e+07
Burnley         -8.145224e+06
Cambridge        1.656442e+06
Chatham         -7.676703e+06
Coventry        -9.926826e+06
Crawley         -1.023227e+05
Derby           -9.186247e+06
Doncaster       -1.620726e+07
Exeter          -2.126112e+07
Gloucester      -7.437181e+06
Huddersfield    -1.358214e+07
Hull            -2.819193e+07
Ipswich         -6.117255e+06
Leeds           -2.016163e+07
Leicester       -2.252432e+07
Liverpool       -3.188233e+07
London          -8.132402e+08
Luton           -1.833219e+06
Manchester      -1.382218e+08
Mansfield       -1.107024e+07
Middlesbrough   -6.264790e+07
Milton Keynes    1.418017e+06
Newcastle       -9.854916e+07
Norwich         -2.664196e+07
Nottingham      -3.308830e+07
Oxford          -1.356891e+06
Peterborough    -9.140605e+06
Plymouth        -6.246374e+07
Ports

In [24]:
y_ij_m_new.to_csv("flow_results.csv",encoding="utf-8")

In [90]:
    flowmapIndex = generate_ij_m_index(city_employment_2017.index, 
                                               city_employment_2017.columns, 
                                               include_national=False)

In [27]:
FlowMapData_with_UK=y_ij_m_new.reset_index()
FlowMapData_with_UK.to_csv("FlowMapData_withUK.csv")

In [28]:
IO.sectors

['Agriculture',
 'Production',
 'Construction',
 'Distribution, transport, hotels and restaurants',
 'Information and communication',
 'Financial and insurance',
 'Real estate',
 'Professional and support activities',
 'Government, health & education',
 'Other services']

In [26]:
FlowMapData=y_ij_m_new[(y_ij_m_new["City"]!="UK") & (y_ij_m_new["Other_City"]!="UK") ].reset_index()
FlowMapData.to_csv("FlowMapData.csv",encoding='utf8')

In [174]:
geolist=gpd.read_file("estios/uk/data/cities_towns.geojson")
import numpy as np
geolist['NAME1']=np.where(geolist['NAME1'] == "Newcastle upon Tyne","Newcastle",geolist['NAME1'])
geolist['NAME1']=np.where(geolist['NAME1'] == "Kingston upon Hull","Hull",geolist['NAME1'])


NameError: name 'gpd' is not defined